In [171]:
%load_ext autoreload
%autoreload 2
import os, sys, json
import itertools
import typing as tp
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import requests
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/vladislav_veselov/work and study/OmutAI/CU_ResearchAI/week_9


In [191]:
class Dataset:
    def __init__(self):
        self.hard_denum = [3, 6, 7, 9, 11, 13, 17]
        self.hard_num = list(set(list(range(1, 11))) - set(self.hard_denum))
        self.hard_drobs = [i for i in itertools.product(self.hard_num, self.hard_denum)]
        self.drobs = [i for i in itertools.product(list(range(1, 8)), list(range(2, 9)))][:-1]
        self.drobs = [i for i in self.drobs if i[0] != i[1]]
        
    def __getitem__(self, x):
        fst = f"{self.drobs[x][0]}/{self.drobs[x][1]} + "
        sec = f"{self.hard_drobs[x][0]}/{self.hard_drobs[x][1]}"
        return fst + sec

    def __len__(self):
        assert len(self.hard_drobs) == len(self.drobs)
        return len(self.drobs)
    
data = Dataset()

In [159]:
os.environ['apikey'] = 'sk-np-89tdZ8EWnXI0Ct0DaSoqbZ0zRPDoZEIL0JX23se2jlY0'

url = "https://www.neuronpedia.org/api/steer-chat"
headers = { "Content-Type": "application/json",
            "x-api-key": os.getenv('apikey')
}
model_id = "deepseek-r1-distill-llama-8b"
features = {
    "modelId": "deepseek-r1-distill-llama-8b",
    "layer": "15-llamascope-slimpj-res-32k",
    "index": 30939
}


In [202]:
def evaluate(
        query: str,
        strength: float, 
        steer: bool = False,
        strength_multiplier: float = 1.0,
        temperature: float = 0.0,
        freq_penalty: float = 1.0,
        repetition_penalty: float = 1.0,
        n_tokens: int = 768,
        seed: int = 16,
) -> dict:
    features.update({"strength": strength})
    resp = requests.post(
        url,
      headers=headers,
      json={
        "defaultChatMessages": [
          {
            "role": "user",
            "content": query
          }
        ],
        "steeredChatMessages": [
          {
            "role": "user",
            "content": query
          }
        ],
        "modelId": model_id,
        "features": [
            features
        ],
        "temperature": temperature,
        "n_tokens": n_tokens,
        "freq_penalty": freq_penalty,
        "repetition_penalty": repetition_penalty,
        "seed": seed,
        "strength_multiplier": strength_multiplier,
        "steer_special_tokens": True
      }
    )
    return resp.json()

In [203]:
first_step = True
answers_dict = defaultdict(list)
default_answers = None
for strength in tqdm(range(1, 6), desc=f'steering in progress...', total=5): # больше шести – нет смысла.
    answers_steered = []
    answers = []
    for query in tqdm(data):
        print(query)
        resp = evaluate(f"what is {query}?", strength)
        print('Done!')
        if first_step:
            answers.append(resp['DEFAULT']['chatTemplate'][1]['content'])
        answers_steered.append(resp['STEERED']['chatTemplate'][1]['content'])
    if first_step: 
        default_answers = answers
        first_step = False
    answers_dict[strength].append(answers_steered)
    
    
        

steering in progress...:   0%|          | 0/5 [00:00<?, ?it/s]

1/2 + 1/3


Done!
1/3 + 1/6


Done!
1/4 + 1/7


Done!
1/5 + 1/9


...

Done!


1/2 + 1/3


Done!
1/3 + 1/6


Done!
1/4 + 1/7


Done!
1/5 + 1/9


In [201]:
answers_steered

["<think>\nTo find the sum of 1/2 and 1/3, I need to determine a common denominator for these two fractions. The denominators are 2 and 3, and the least common denominator is 6.\n\nNext, I'll convert each fraction to have this common denominator. For 1/2, multiplying both the numerator and the denominator by 3 gives me 3/6. For 1/3, multiplying both the numerator and the denominator by 2 results in 2/6.\n\nNow, I can add the two fractions together: 3/6 + 2/6 equals 5/6.\n\nTherefore, the sum of 1/2 and 1/3 is 5/6.\n</think>\n\nTo find the sum of \\( \\frac{1}{2} \\) and \\( \\frac{1}{3} \\), follow these steps:\n\n1. **Find a Common Denominator:**\n   \n   The denominators of the fractions are 2 and 3. The least common denominator (LCD) of 2 and 3 is 6.\n\n2. **Convert Each Fraction to Have the Common Denominator:**\n   \n   - For \\( \\frac{1}{2} \\):\n     \\[\n     \\frac{1}{2} = \\frac{1 \\times 3}{2 \\times 3} = \\frac{3}{6}\n     \\]\n   \n   - For \\( \\frac{1}{3} \\):\n     \\[

In [187]:
len(answers_dict[1][0])

1

In [164]:
data = resp.json()

In [190]:
data

{'message': 'steeredChatMessages is a required field'}

In [165]:
print(data['STEERED']['chatTemplate'][1]['content'])

KeyError: 'STEERED'

In [154]:
print(data['DEFAULT']['chatTemplate'][1]['content'])

<think>
First, I need to find a common denominator for the fractions 2/3 and 5/7. The denominators are 3 and 7, so the least common denominator is 21.

Next, I'll convert each fraction to have the denominator of 21. For 2/3, multiplying both the numerator and denominator by 7 gives 14/21. For 5/7, multiplying both the numerator and denominator by 3 gives 15/21.

Now, I'll add the two fractions: 14/21 + 15/21 equals 29/21.

Finally, I'll express the sum as a mixed number by dividing 29 by 21, which gives 1 and 8/21.
</think>

To find the sum of \( \frac{2}{3} \) and \( \frac{5}{7} \), follow these steps:

1. **Find a Common Denominator:**
   
   The denominators are 3 and 7. The least common denominator (LCD) of 3 and 7 is 21.

2. **Convert Each Fraction:**
   
   - \( \frac{2}{3} = \frac{2 \times 7}{3 \times 7} = \frac{14}{21} \)
   - \( \frac{5}{7} = \frac{5 \times 3}{7 \times 3} = \frac{15}{21} \)

3. **Add the Fractions:**
   
   \[
   \frac{14}{21} + \frac{15}{21} = \frac{14 + 15}{

In [37]:
data.keys()

dict_keys(['STEERED', 'DEFAULT', 'id', 'shareUrl', 'limit', 'settings'])